In [1]:
import pandas as pd

Датасет [ссылка](https://www.kaggle.com/datasets/rabieelkharoua/students-performance-dataset)

В нем есть GPA и GradeClass, оба этих столбца это оценка, но GPA это вещественное значение, которое можно использовать для регрессии, а GradeClass можно использовать для классфикации.

В нем нет пропусков данных.

In [33]:
df = pd.read_csv("student_performance.csv")

In [30]:
len(df)

2392

Первое, что мы сделаем, будет классификация

In [34]:
df_class = df.drop(columns=["Ethnicity", "StudentID", "GPA"])

In [35]:
X = df_class.drop(["GradeClass"], axis=1)
y = df_class["GradeClass"]

In [5]:
X.head()

,Age,Gender,ParentalEducation,StudyTimeWeekly,Absences,Tutoring,ParentalSupport,Extracurricular,Sports,Music,Volunteering
0,17,1,2,19.833723,7,1,2,0,0,1,0
1,18,0,1,15.408756,0,0,1,0,0,0,0
2,15,0,3,4.210570,26,0,2,0,0,0,0
3,17,1,3,10.028829,14,0,3,1,0,0,0
4,17,1,2,4.672495,17,1,3,0,0,0,0


In [6]:
y.head()

0    2.0
1    1.0
2    4.0
3    3.0
4    4.0
Name: GradeClass, dtype: float64

In [7]:
from sklearn.model_selection import train_test_split

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [9]:
from sklearn.ensemble import RandomForestClassifier

In [42]:
rf = RandomForestClassifier(
    criterion="entropy",
)
rf.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy')

In [11]:
from sklearn.metrics import classification_report

In [43]:
y_pred = rf.predict(X_test)
print("Score", rf.score(X_test, y_test))

Score 0.6597077244258872


In [44]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.67      0.22      0.33        27
         1.0       0.44      0.47      0.46        49
         2.0       0.48      0.49      0.49        85
         3.0       0.53      0.39      0.45        99
         4.0       0.80      0.94      0.87       219

    accuracy                           0.66       479
   macro avg       0.58      0.50      0.52       479
weighted avg       0.64      0.66      0.64       479



In [45]:
features = pd.DataFrame(rf.feature_importances_, index=X_test.columns)
print(features.head(15))

                          0
Age                0.061516
Gender             0.031136
ParentalEducation  0.060849
StudyTimeWeekly    0.177902
Absences           0.468249
Tutoring           0.026216
ParentalSupport    0.071786
Extracurricular    0.027317
Sports             0.030169
Music              0.023559
Volunteering       0.021302


Теперь попробуем классификацию с кастомными параметрами

In [46]:
rf = RandomForestClassifier(
    n_estimators=1000,
    criterion="entropy",
    min_samples_split=10,
    max_depth=14,
)
rf.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', max_depth=14, min_samples_split=10,
                       n_estimators=1000)

In [47]:
y_pred = rf.predict(X_test)
print("Score", rf.score(X_test, y_test))

Score 0.6805845511482255


In [48]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.75      0.11      0.19        27
         1.0       0.50      0.55      0.52        49
         2.0       0.55      0.55      0.55        85
         3.0       0.59      0.41      0.49        99
         4.0       0.78      0.95      0.86       219

    accuracy                           0.68       479
   macro avg       0.64      0.52      0.52       479
weighted avg       0.67      0.68      0.65       479



In [49]:
features = pd.DataFrame(rf.feature_importances_, index=X_test.columns)
print(features.head(15))

                          0
Age                0.041747
Gender             0.020292
ParentalEducation  0.042823
StudyTimeWeekly    0.139538
Absences           0.580537
Tutoring           0.028903
ParentalSupport    0.065014
Extracurricular    0.022654
Sports             0.024791
Music              0.017638
Volunteering       0.016061


Теперь посмотрим на регрессию

In [50]:
df_regr = df.drop(columns=["Ethnicity", "StudentID", "GradeClass"])

In [51]:
X = df_regr.drop(["GPA"], axis=1)
y = df_regr["GPA"]

In [52]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [22]:
from sklearn.ensemble import RandomForestRegressor

In [53]:
rf = RandomForestRegressor()
rf.fit(X_train, y_train)

RandomForestRegressor()

In [54]:
from sklearn.metrics import (
    mean_absolute_error,
    mean_squared_error,
    r2_score,
)

In [55]:
y_pred = rf.predict(X_test)
print("Score", rf.score(X_test, y_test))
print("MAE", mean_absolute_error(y_test, y_pred))
print("MSE", mean_squared_error(y_test, y_pred))
print("R2", r2_score(y_test, y_pred))

Score 0.9201300793667067
MAE 0.19679344170890142
MSE 0.06415520684234921
R2 0.9201300793667067


In [56]:
features = pd.DataFrame(rf.feature_importances_, index=X_test.columns)
print(features.head(15))

                          0
Age                0.006821
Gender             0.002895
ParentalEducation  0.006808
StudyTimeWeekly    0.060489
Absences           0.856398
Tutoring           0.012481
ParentalSupport    0.032358
Extracurricular    0.008833
Sports             0.007329
Music              0.003458
Volunteering       0.002130
